In [33]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='theano'
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
%matplotlib inline
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']
docss=['a','b','c','d','e','f','g','h']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]

In [34]:
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(docs)

sequence=tokenizer.texts_to_sequences(docs)
print(sequence)
print(tokenizer.word_index)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]
{'work': 1, 'done': 2, 'good': 3, 'effort': 4, 'poor': 5, 'well': 6, 'great': 7, 'nice': 8, 'excellent': 9, 'weak': 10, 'not': 11, 'could': 12, 'have': 13, 'better': 14}


In [35]:
vocab_size=50
#on_hot原理，即把所有的单词转为长度为n的向量，为了方便起见，只将1的位置记住
encoded_docs=[one_hot(d,vocab_size) for d in docs]
print(encoded_docs)

[[9, 8], [18, 12], [41, 21], [45, 12], [30], [28], [10, 21], [12, 18], [10, 12], [16, 23, 8, 2]]


In [36]:
from keras.utils.np_utils import to_categorical
data=pad_sequences(sequence,4)
# print(data)
labels=to_categorical(labels,10)
# print(labels)

In [37]:
import numpy as np
embedding_matrix = np.ones((len(tokenizer.word_index) + 1, 10))
print(embedding_matrix)
# print(embedding_matrix)
# for word,i in tokenizer.word_index.items():
#     print(word)
#     print(i)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [38]:
embedding_layer=Embedding(len(tokenizer.word_index) + 1,
                            10,
                            weights=[embedding_matrix],
                            input_length=4,
                            trainable=True)
print(embedding_layer.get_config())

{'name': 'embedding_8', 'trainable': True, 'batch_input_shape': (None, 4), 'dtype': 'float32', 'input_dim': 15, 'output_dim': 10, 'embeddings_initializer': {'class_name': 'RandomUniform', 'config': {'minval': -0.05, 'maxval': 0.05, 'seed': None}}, 'embeddings_regularizer': None, 'activity_regularizer': None, 'embeddings_constraint': None, 'mask_zero': False, 'input_length': 4}


In [42]:
sequence_input=Input(shape=(4,),dtype='int32')
embedding_sequences=embedding_layer(sequence_input)
# print(embedding_sequences)
l_lstm=Bidirectional(LSTM(100))(embedding_sequences)
preds=Dense(10,activation='softmax')(l_lstm)
model=Model(sequence_input,preds)
model.compile(loss='categorical_crossentropy'
              ,optimizer='rmsprop',
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 4, 10)             150       
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               88800     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2010      
Total params: 90,960
Trainable params: 90,960
Non-trainable params: 0
_________________________________________________________________
